# Оценка скорости закрытия 

- Можно взять общее время затраченное на решение всех инцидентов как t, общщее количестов инцидентов, как n и посчитать количество инцидинетов в среднем решаемых за день `inc_speed = n/t`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ACCOUNT_ID = 1

### Загружаем нужные нам таблицы

In [ ]:
data_inc = pd.read_csv('./data/mysql_table_incidents.csv')
data_inc_types = pd.read_csv('./data/mysql_table_incident_types.csv')

### Фильтруем по нужным нам 1 или 2 аккаунту

In [ ]:
data_query_inc = data_inc[(data_inc['account_id'] == ACCOUNT_ID)]    # | (data_inc['account_id'] == 2)

In [ ]:
#берем только закрытые инциденты
data_query_inc_closed = data_query_inc[(data_query_inc['closed_at'].isna() == False)]

### Хватает ли у нас данных для расчета среднего времени по отделным типам инцедентов?

In [ ]:
data_query_inc_closed['incident_type_id'].value_counts()
#Делаем топ-5 инцедентов и по ним считаем среднюю оценку закрытия

In [ ]:
data_query_inc_closed['incident_type_id'].value_counts().keys()[:5]

In [ ]:
#расчёт времени, затраченного на решение инцидентов,
data_query_inc_closed['close_time'] = pd.to_datetime(data_query_inc_closed['closed_at']) - pd.to_datetime(data_query_inc_closed['created_at'])

### Расчёт среднего и медианного значений закрытия времени инцидентов у

In [ ]:
all_average = data_query_inc_closed['close_time'].mean()
all_median = data_query_inc_closed['close_time'].median()

stats_days_med = all_median.total_seconds() /60 /60 /24
stats_days_aver = all_average.total_seconds() /60 /60 /24

all_inc_dict = {
    'median': round(stats_days_med, 2),
    'mean': round(stats_days_aver, 2)
}

# Для BACKEND!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
all_inc_dict

# Считаем среднее и медианное по каждому из топ-5 инцидентов

In [ ]:
data_query_inc_closed['delta_days'] = data_query_inc_closed['close_time'].dt.total_seconds() /60 /60 /24

dict_inc_means = data_query_inc_closed.groupby(['incident_type_id']).mean()['delta_days']
dict_inc_medians = data_query_inc_closed.groupby(['incident_type_id']).median()['delta_days']

In [ ]:
TOP_NUM = len(data_query_inc.incident_type_id.unique())

top_types_id = data_query_inc_closed['incident_type_id'].value_counts().keys()[:TOP_NUM]
column_title = []
top_dict_means = {}
top_dict_medians = {}

for idx in top_types_id:
    string = data_inc_types[data_inc_types['id'] == idx].title
    string = string.to_string().split('    ')[1]
    column_title.append(string)

    top_dict_means[string] = round(dict_inc_means.get(idx), 2)
    top_dict_medians[string] = round(dict_inc_medians.get(idx), 2)

# Для BACKEND!!!!!!!!!!!!!!!!!

In [ ]:
top_dict_means

In [ ]:
top_dict_medians

# Переходим к визуализации результатов

In [ ]:
y = list(top_dict_means.keys())
x = list(top_dict_means.values())
plt.figure(figsize=(15, 7))

plt.title(f'Затраченное время на решение инцидентов')
plt.xticks(np.arange(round(min(x)), round(max(x)) + 1, step=10))
plt.grid()
plt.xlabel('Количество дней, затраченных на решение инцидента')
plt.ylabel('Тип инцидента', color='white')
plt.barh(y, x, alpha=0.5, label='Среднее затраченное время')
plt.axvline(all_inc_dict['mean'], color='green', linewidth=4, label=f'Среднее время решения среди всех инцидентов:{all_inc_dict.get("mean")}')

y_new = list(top_dict_medians.keys())
x_new = list(top_dict_medians.values())

plt.barh(y_new, x_new, color='orange', alpha=0.5, label='Медианное затраченное время')
plt.axvline(all_inc_dict['median'], color='red', linewidth=4, label=f'Медианное время решения среди всех инцидентов:{all_inc_dict.get("median")}')
plt.legend()
plt.tight_layout()
plt.savefig('./results/median_mean_inc.jpg', dpi=1200)

plt.show()

In [ ]:
time = data_query_inc_closed['delta_days'].sum()
num_inc = data_query_inc_closed.size

inc_per_day = round(num_inc / time, 2)

In [ ]:
inc_per_day

# Комплексная оценка: inc_per_day = инцедентов/день

top_dict_means, top_dict_medians = затраченное время на решение инцидентов каждого типа